In [ ]:
#libraries

import pandas as pd  

import numpy as np            

from scipy.optimize import minimize    

from numba import jit


import plotly
import plotly.express as px

## Model

In [ ]:
# the decorator from numba is for 3X speed in fitting

@jit()  
def loglike(price: float, count: int, alfa: float, beta: float) -> float:
    
    '''
    Log-Likelihood from demand exponential function:
    
    demand = np.exp(alfa*price + beta)
    
    Args:
    price : float, product price
    count : int, product quantity sold in that price
    alfa : float, first fit parameter, price parameter
    beta : float, second fit parameter, intercept parameter
    
    
    Return:
    log-likelihood : float 
    '''
    
    demand = np.exp(alfa*price + beta)
    
    return -demand + count*np.log(demand) 

In [ ]:
def fit(X: np.array) -> np.array:
    
    '''
    Fit model function. 
    
    Args:
    X : np.array with shape (N, 2), first column is price and the second is count_
    
    Return:
    params : np.array with two float, alfa and beta
    '''
    
    
    # array of alfa and beta initial values 
    initial_values = np.array([-1., 0.]) # alfa, beta               
  

    def minus_loglike(params):  # Log-Likelihood, function to be minimize
        
        '''
        Args:
        params : np.array with two float, alfa and beta
        
        Return:
        log-likelihood : float 
        '''
        
        
        log_like = map(lambda elem: loglike(elem[0],     # price from data
                                            elem[1],     # count from data
                                            params[0],   # alfa parameter
                                            params[1]),  # beta parameter
                       X)
        
        
        
        return -sum(log_like) 
        
    
    output=minimize(minus_loglike, 
                    initial_values, 
                    options={'disp':True, 'maxiter':10000}, 
                    method='trust-constr') 

    return output.x   # alfa, beta

In [ ]:
def predict(params: np.array) -> list:
    
    '''
    Function to predict demand & profit functions from parameters.
    
    Args:
    params : np.array with two float, alfa and beta
    
    Return:
    demand & profit functions : list with two np.array with 100 ordered values each
    '''
    
    x = np.linspace(1, 100, 100)
    
    demand = np.exp(params[0]*x + params[1])
    
    profit = x * demand * 100
    
    return [demand, profit]

## Data

In [ ]:
data = pd.read_parquet('data.parquet')

data.head()

In [ ]:
data.info(memory_usage='deep')

In [ ]:
data.describe(datetime_is_numeric=True)

## Fit and predict

In [ ]:
X = data[['price', 'count_']]

X.head()

In [ ]:
X = X.values

X[0]   # price, count

In [ ]:
%%time

params = fit(X)

params

In [ ]:
prediction = predict(params)

## Plots

In [ ]:
df_plot=pd.DataFrame()

df_plot['x'] = np.linspace(1, 100, 100)

df_plot['demand'] = prediction[0]

df_plot['profit'] = prediction[1]

df_plot.head()

In [ ]:
# demand plot

px.line(df_plot, x='x', y='demand')

In [ ]:
# profit plot

px.line(df_plot, x='x', y='profit')